In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from nltk import pos_tag

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sarthak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('twitter_x_y_train.csv')#Training Data 
dft = pd.read_csv('twitter_x_test.csv')#Testing Data

df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
df = df[['airline_sentiment','text']]
dft = dft[['text']]
print(dft.head())
df.head()

                                                text
0  @AmericanAir In car gng to DFW. Pulled over 1h...
1  @AmericanAir after all, the plane didn’t land ...
2  @SouthwestAir can't believe how many paying cu...
3  @USAirways I can legitimately say that I would...
4  @AmericanAir still no response from AA. great ...


,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...


In [5]:
data = np.array(df.text)
target = np.array(df.airline_sentiment)
data_test = np.array(dft.text)

In [39]:
data_test[:5]

array(["@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?",
       '@AmericanAir after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs.',
       "@SouthwestAir can't believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow.",
       '@USAirways I can legitimately say that I would have rather driven cross country than flown on US Airways.',
       '@AmericanAir still no response from AA. great job guys!'],
      dtype=object)

In [8]:
Tok_data_train = []
for i in range(len(data)):
    Tok_data_train.append(word_tokenize(data[i]))
    
Tok_data_test = []
for i in range(len(data_test)):
    Tok_data_test.append(word_tokenize(data_test[i]))

In [9]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
#stops, string.punctuation

In [12]:
def clean_tweets(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [13]:
cleaned = []
for words in Tok_data_train:
    cleaned.append(clean_tweets(words))
    

cleaned_test = []
for words in Tok_data_test:
    cleaned_test.append(clean_tweets(words))

In [40]:
text_train = [" ".join(words) for words in cleaned]
text_test = [" ".join(words) for words in cleaned_test]

text_test[:5]

["americanair car gng dfw pulled 1hr ago icy road on-hold aa since 1hr ca n't reach arpt aa2450 wat 2",
 'americanair plane ’ land identical bad condition grk accord metars',
 "southwestair ca n't believe many pay customer left high dry reason flight cancelled flightlations monday bdl wow",
 'usairways legitimately say would rather driven cross country flown us airways',
 'americanair still response aa great job guy']

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_train, target)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(max_features = 2000, ngram_range=(1,2))
x_train_features = count_vec.fit_transform(x_train)
#x_test_features = count_vec.fit_transform(text_documents_test)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [45]:
count_vec.get_feature_names()[200:250]

['appease',
 'apply',
 'appreciate',
 'april',
 'area',
 'around',
 'arrival',
 'arrive',
 'arrives',
 'as',
 'asap',
 'ask',
 'assign',
 'assist',
 'assistance',
 'atl',
 'atlanta',
 'attempt',
 'attendant',
 'attitude',
 'austin',
 'auto',
 'automate',
 'automatically',
 'avail',
 'available',
 'avgeek',
 'avoid',
 'award',
 'aware',
 'away',
 'awesome',
 'awful',
 'baby',
 'back',
 'back gate',
 'back home',
 'back late',
 'bad',
 'bad airline',
 'bad customer',
 'bad experience',
 'bad service',
 'bad weather',
 'badcustomerservice',
 'badservice',
 'bag',
 'bag flight',
 'bag lose',
 'bag make']

In [18]:
x_test_features = count_vec.transform(x_test)

In [19]:
x_test_features

<2745x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 26139 stored elements in Compressed Sparse Row format>

Support Vector Classifier 

In [20]:
from sklearn.svm import SVC

In [21]:
svc = SVC(C = 2.0)
svc.fit(x_train_features, y_train)

SVC(C=2.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
svc.score(x_test_features, y_test)

0.6123861566484518

Random Forest Classifier

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rfc = RandomForestClassifier(min_samples_split=4, min_samples_leaf=2)
rfc.fit(x_train_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(x_test_features, y_test)

0.7307832422586521

Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
lr = LogisticRegression()
lr.fit(x_train_features, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
lr.score(x_test_features, y_test)

0.775591985428051

Decision Tree Classifier

In [31]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [32]:
dtc.fit(x_train_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [33]:
dtc.score(x_test_features, y_test)

0.6739526411657559

So I can see that Logistic Regression is giving best results as of now without manipulating the parameters of classifiers. So I must predict the classes of testing data using Logistic Regression:

In [36]:
test_features = count_vec.transform(text_test)

In [37]:
p = lr.predict(test_features)

In [38]:
p

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'neutral'], dtype=object)